# Generative Adversarial Network

Dieses Jupyter-Notebook-Dokument enthält alles, was zum Trainieren eines generativen adversen Netzwerks erforderlich ist, um Bilder aus dem MNINST- und CIFAR10-Datensatz zu erstellen.

### Libraries

##### Standardbibliotheken
- os: ermöglicht Erstellung von Ordnern und Navigation durch die Ordnerstruktur des Projektes
- sys: sys.path.append erlaubt das Importieren von Klassen, welche außerhalb definiert sind (im src-Ordner)
- torch & torch.nn: Standard Pytorch Klassen
- torchvision transforms: Transformation von Tensoren
- torchvision.utils save_image: ermöglicht das Abspeichern der Bilder die erzeugt werden


##### Custom Imports
- classes.gan.XY: importiert die jeweiligen GAN Klassen die im Ordner src/classes/gan definiert sind.
- Klassen / Funktionen aus dem Ordner src/utils/ sind selbsterstelle Klassen / Funktionen, welche Code kapseln und so die Lesbarkeit verbessern. Die Dokumentation zu den Klassen / Funktionen findet sich in den entsprechenden Dateien


In [ ]:
import os
import sys
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

import datetime

# custom imports
sys.path.append('../src')
from utils.dataset import load_datasets
from utils.foldergen import generate_folder
from classes.gan.gan import Generator, Discriminator
from classes.gan.ganCNN import GeneratorCNN, DiscriminatorCNN



### Hyperparameters

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

datasettype = "MNIST" # only option is "MNIST"
networktype = "NN" # select either "NN" or "CNN"

# Train a new network or continue training a previously trained network:
continueTraining = False;

# learning rate
lr = 0.0002

# number of epochs
num_epochs = 100
batch_size = 25

hidden_dim = 100

### Get Data

Im Folgenden wird die ``transform``-Funktion definiert, mit der die Daten transformiert werden.

Danach werden die Ordner erstellt, wo nachher der Output landet, falls diese noch nicht existieren.

Zum Schluss werden zwei Datasets und zwei Dataloader erzeugt, einmal mit dem Trainingsdatensatz, einmal mit dem Testdatenssatz.

In [ ]:
if datasettype == "MNIST":
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(
                [0.5], [0.5]
            ),
        ]
    )
elif datasettype == "CIFAR10":
    transform = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

# create folder structure
load_folder = "load"
output_folder = "output"
folders = [load_folder, output_folder]
generate_folder(folders)

dataset_train, loader_train = load_datasets(datasettype, transform, batch_size)
dataset_test, loader_test = load_datasets(datasettype, transform, batch_size, train=False, download=False)

### Network

Der Generator wird als ``Gen`` und der Discriminator als ``Dis`` initialisiert.
Die Initialisierung ist abhängig von dem oben gesetzen ``networktype``.

Als ``lossFunction`` wird Pytorchs BCELoss() Funktion genutzt.

Zum Schluss werden zwei Adam-Optimizer erstellt, jeweils einer für den Generator und einen für den Diskriminator.

In [ ]:
z_dim = 100

# set image_channels depending on datasettype
if datasettype == "MNIST":
    image_channels = 1
    data_dim = 28 * 28
elif datasettype == "CIFAR10":
    image_channels = 3
    data_dim = 32 * 32

if networktype == "NN":
    Gen = Generator(g_input_dim= z_dim, g_hidden_dim= hidden_dim, g_output_dim= data_dim * image_channels).to(device)
    Dis = Discriminator(d_input_dim= data_dim * image_channels,  d_hidden_dim= hidden_dim).to(device)
elif networktype == "CNN":
    Gen = GeneratorCNN(latent_dim = z_dim, img_channels = image_channels, img_size = data_dim)
    Dis = DiscriminatorCNN(img_channels = image_channels, img_size = data_dim)

lossFunction = nn.BCELoss()

# Optimizers
Gen_optimizer = torch.optim.Adam(Gen.parameters(), lr=lr)
Dis_optimizer = torch.optim.Adam(Dis.parameters(), lr=lr)

### Training

Im folgenden wird eine Trainingsstep für den Diskriminator beschrieben.

Dabei wird zunächst ein Forward-Pass mit echten Bildern aus dem Datensatz durchgeführt. Danach wird der Loss des Diskriminators mit den echten Daten berechnet.

Dann wird ein Forward-Pass mit den künstlichen Daten des Generators durchgeführt. Dann wird der Loss des Diskriminators mit den künstlichen Daten berechnet.

Beide Losses werden addiert und die Backpropagation findet statt.

In [ ]:
def Dis_train(x):
    Dis.zero_grad()

    # flatten tensor if a NN is being used
    if networktype == "NN":
        x = x.view(-1, data_dim * image_channels)
    # real data
    y_real = torch.ones(batch_size, 1)
    x_real, y_real = x.to(device), y_real.to(device)

    D_output = Dis(x_real)
    D_real_loss = lossFunction(D_output, y_real)

    # fake data
    z =  torch.randn(batch_size, z_dim).to(device)
    x_fake, y_fake = Gen(z), torch.zeros(batch_size, 1).to(device)

    D_output = Dis(x_fake)
    D_fake_loss = lossFunction(D_output, y_fake)

    # loss
    D_loss = D_real_loss + D_fake_loss
    D_loss.backward()
    Dis_optimizer.step()

    return D_loss.data.item()

Im Folgenden wird ein Trainingsstep für den Generator beschrieben.

Dabei wird dem Generator eine Zufallsmatrix gegeben, daraus erzeugt der Generator ein Bild.

Dieses wird dem Diskriminator gegeben. Der Loss berechnet sich daraus, ob der bzw. wie gut Diskriminator getäuscht werden konnte.

In [ ]:
def Gen_train(x):
    Gen.zero_grad()

    z = torch.randn(batch_size, z_dim).to(device)
    y = torch.ones(batch_size, 1).to(device)

    G_output = Gen(z)
    D_output = Dis(G_output)
    G_loss = lossFunction(D_output, y)

    # loss
    G_loss.backward()
    Gen_optimizer.step()

    return G_loss.data.item()

### Load model for further training

Im Folgenden werden bereits trainierte Modelle des Diskriminators und Generators geladen.

In [ ]:
# Load model for further training
discriminator_file = "discriminator.pth"
generator_file = "generator.pth"

# Check if directory and files for discriminator and generator exist
if os.path.exists(load_folder) and os.path.isfile(os.path.join(load_folder, discriminator_file)) and os.path.isfile(os.path.join(load_folder, generator_file)) and continueTraining:
    Gen = Generator(g_input_dim = z_dim, g_hidden_dim=hidden_dim, g_output_dim = data_dim * image_channels).to(device)
    Gen.load_state_dict(torch.load(os.path.join(load_folder, generator_file), map_location=device))

    Dis = Discriminator(d_input_dim= data_dim, d_hidden_dim= hidden_dim).to(device)
    Dis.load_state_dict(torch.load(os.path.join(load_folder, discriminator_file), map_location=device))

### Training

Im Folgenden findet das Training der Modelle statt, indem die Trainingssteps des Diskriminators und des Generators aufgerufen werden.

Dabei werden die Ergebnisse, also die Bilder und die neuen Modelle in einem Unterornder gespeichert, welcher das Datum und die Uhrzeit tragen. So soll vermieden werden, dass die gespeicherten Modelle und erzeugten Bilder eventuell überschrieben werden.

In [ ]:
# create folder structure if it does not exist
current_time = datetime.datetime.now()
formatted_time = current_time.strftime("%d%m-%H%M")

# Create subfolders if they don't exist
pictures_folder = os.path.join(output_folder, formatted_time, "pic")
model_folder = os.path.join(output_folder, formatted_time, "model")

if not os.path.exists(pictures_folder):
    os.makedirs(pictures_folder)

if not os.path.exists(model_folder):
    os.makedirs(model_folder)

# Training loop
for epoch in range(1, num_epochs+1):
    D_losses, G_losses = [], []
    for batch_idx, (x, _) in enumerate(loader_train):
        D_losses.append(Dis_train(x))
        G_losses.append(Gen_train(x))

    print('[%d/%d]: loss_d: %.3f, loss_g: %.3f' % (
            (epoch), num_epochs, torch.mean(torch.FloatTensor(D_losses)), torch.mean(torch.FloatTensor(G_losses))))
    
    with torch.no_grad():
        if epoch % 1 == 0:
            test_z = torch.randn(batch_size, z_dim).to(device)
            generated = Gen(test_z)

            # format output string
            formatted_number = "{:0{}}".format(epoch, len(str(num_epochs)))

            if datasettype == "CIFAR10":
                save_image(generated.view(generated.size(0), image_channels, 32, 32), pictures_folder + '/' + formatted_number + '.png')
            elif datasettype == "MNIST":
                save_image(generated.view(generated.size(0), image_channels, 28, 28), pictures_folder + '/' + formatted_number + '.png')

### Save Models

In [ ]:
torch.save(Gen.state_dict(), model_folder + '/' + 'generator.pth')
torch.save(Dis.state_dict(), model_folder + '/' + 'discriminator.pth')